# Trading Environment Test Notebook

This notebook demonstrates and tests the basic operations of the `stock` trading environment class.

## 1. Set Working Directory

Ensure the working directory is set to the main project folder.

In [1]:
import os
import sys

# Change working directory to the parent (main) directory
current_dir = os.getcwd()
print(f"Current Working Directory: {current_dir}")
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
os.chdir(parent_dir)
current_dir = os.getcwd()
print(f"Current Working Directory: {current_dir}")
# We need to be in the main directory that contains tests, models, etc folders

Current Working Directory: /mnt/c/Users/ASUS/OneDrive - SISSA/study/research/neural theory/data_science_finance/deepQN_trading/test_notebooks
Current Working Directory: /mnt/c/Users/ASUS/OneDrive - SISSA/study/research/neural theory/data_science_finance/deepQN_trading


## 2. Import Trading Environment Class

In [2]:
from trading_env_class import stock

## 3. Create Stock Instance and Prepare Test Data

In [3]:
# Create a stock instance with a sample ticker
test_stock = stock("^GSPC")

# Define a test date and price using test_stock.all_dates
test_date = test_stock.all_dates[0]  # Use the first date in all_dates
test_price = test_stock.all_prices[0]

## 4. Test Buy and Sell Operations

The following cells perform a sequence of buy and sell operations and print the immediate and realized profits to verify correctness.

**Additional checks:**
- Test both buy and sell (long and short) operations.
- Test realized profit after closing all positions.
- Test that immediate profit is updated correctly after each operation.
- Test edge cases: selling more than owned, buying with no position, closing with no open position.

### Test 1: Short selling and covering

In [17]:
test_stock = stock("^GSPC")
test_date = test_stock.all_dates[0]
test_price = test_stock.all_prices[0]

# Short 10 shares
test_stock.sell(test_date, 10)
# Immediate profit should be zero after opening short (no closing action yet)
assert test_stock.immediate_profit == 0, "Immediate profit should be zero after opening short."
assert test_stock.realized_profit == 0, "Realized profit should be zero after opening short."

In [18]:
# Cover 5 shares next day
test_date2 = test_stock.all_dates[1]
cover_price = test_stock.all_prices[1]
test_stock.buy(test_date2, 5)
# Immediate profit is realized only for the closed portion (5 shares)
expected_profit = 5 * (test_price - cover_price)
assert abs(test_stock.immediate_profit - expected_profit) < 1e-8, "Immediate profit after partial cover incorrect."
assert abs(test_stock.realized_profit - expected_profit) < 1e-8, "Realized profit after partial cover incorrect."

In [19]:
# Cover remaining 5 shares
test_date3 = test_stock.all_dates[2]
cover_price2 = test_stock.all_prices[2]
test_stock.buy(test_date3, 5)
expected_profit2 = 5 * (test_price - cover_price2)
assert abs(test_stock.immediate_profit - expected_profit2) < 1e-8, "Immediate profit after full cover incorrect."
assert abs(test_stock.realized_profit - (expected_profit + expected_profit2)) < 1e-8, "Realized profit after full cover incorrect."

In [20]:
# Realized profit should match sum of all immediate profits after all positions closed
test_stock.close(test_date3)
assert test_stock.short_position == 0 and test_stock.long_position == 0, "All positions should be closed."
print("Short selling test passed.")

Short selling test passed.


### Test 2: Long buying and selling

In [21]:
test_stock = stock("^GSPC")
test_date = test_stock.all_dates[0]
buy_price = test_stock.all_prices[0]
test_stock.buy(test_date, 10)
# Immediate profit should be zero after opening long (no closing action yet)
assert test_stock.immediate_profit == 0, "Immediate profit should be zero after opening long."
assert test_stock.realized_profit == 0, "Realized profit should be zero after opening long."

In [22]:
test_date2 = test_stock.all_dates[1]
sell_price1 = test_stock.all_prices[1]
test_stock.sell(test_date2, 5)
expected_profit = 5 * (sell_price1 - buy_price)
assert abs(test_stock.immediate_profit - expected_profit) < 1e-8, "Immediate profit after partial sell incorrect."
assert abs(test_stock.realized_profit - expected_profit) < 1e-8, "Realized profit after partial sell incorrect."

In [23]:
test_date3 = test_stock.all_dates[2]
sell_price2 = test_stock.all_prices[2]
test_stock.sell(test_date3, 5)
expected_profit2 = 5 * (sell_price2 - buy_price)
assert abs(test_stock.immediate_profit - expected_profit2) < 1e-8, "Immediate profit after full sell incorrect."
assert abs(test_stock.realized_profit - (expected_profit + expected_profit2)) < 1e-8, "Realized profit after full sell incorrect."

In [24]:
test_stock.close(test_date3)
assert test_stock.short_position == 0 and test_stock.long_position == 0, "All positions should be closed."
print("Long buying test passed.")

Long buying test passed.


### Test 3: Edge case - closing with no open position

In [25]:
test_stock = stock("^GSPC")
test_date = test_stock.all_dates[0]
test_stock.close(test_date)
# No trades made, so realized profit should be zero and positions should be closed
assert test_stock.realized_profit == 0, "Realized profit should be zero if no trades were made."
assert test_stock.short_position == 0 and test_stock.long_position == 0, "All positions should be closed if no trades were made."
print("Edge case (close with no position) test passed.")

Edge case (close with no position) test passed.
